In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# url of Ist page

url = 'https://www.themoviedb.org/movie?page-' + str(1)

headers = {"user-agent": 
           "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

response = requests.get(url, headers = headers)

In [3]:
doc = BeautifulSoup(response.text, 'html.parser')

In [4]:
# getting name of the movie

Name_tags = doc.find_all('h2')
Movies_names = []
for i in range (4, len(Name_tags)) :
     Movies_names.append(Name_tags[i].text)

In [5]:
# release year of movie

release_tags = doc.find_all('p')
release_year = []
for i in range (1, 21):
     release_year.append(release_tags[i].text)

In [6]:
# Link of movie

link_tags = doc.find_all('a', {'class' : 'image'})
movie_links = []
base_url ='https://www.themoviedb.org'
for i in range (len(link_tags)):
 
 movie_links.append(base_url + link_tags[i]['href'])

In [7]:
# rating of movie

rating = []
rating_tag = doc.find_all('div', {'class' : 'user_score_chart'})
for i in range (len(rating_tag)):
     rating.append(rating_tag[i]['data-percent'])

In [8]:
# Following same procedure as above for 500 pages

for i in range(2,501):
    
    url = 'https://www.themoviedb.org/movie?page-' + str(i)
    
    response = requests.get(url, headers = headers)
    
    doc = BeautifulSoup(response.text, 'html.parser')
    
    Name_tags = doc.find_all('h2')
    for i in range (4, len(Name_tags)) :
         Movies_names.append(Name_tags[i].text)
            
    release_tags = doc.find_all('p')
    for i in range (1, 21):
         release_year.append(release_tags[i].text)
            
    link_tags = doc.find_all('a', {'class' : 'image'})
    base_url ='https://www.themoviedb.org'
    for i in range (len(link_tags)):
         movie_links.append(base_url + link_tags[i]['href'])
            
    rating_tag = doc.find_all('div', {'class' : 'user_score_chart'})
    for i in range (len(rating_tag)):
         rating.append(rating_tag[i]['data-percent'])

In [9]:
len(movie_links)

10000

# Getting data about each movie using movies links

In [10]:
Movie_url = movie_links[0]
r = requests.get(Movie_url, headers = headers)
movie_docs = BeautifulSoup(r.text, 'html.parser')

In [11]:
# desc about movie
desc_tag = movie_docs.find_all('div', {'class' : 'overview'})
desc = desc_tag[0].text.strip()

In [12]:
# Director of movie
director_tag = movie_docs.find_all('li', {'class' : 'profile'})
Director = director_tag[0].text.strip().split('\n')[0]

In [13]:
# Movie Runtime
runtime_tag = movie_docs.find_all('span', {'class' : 'runtime'})
runtime = runtime_tag[0].text.strip()

In [14]:
# Budget of movie
side_column_tags = movie_docs.find('section', {'class' : 'facts left_column'}).find_all('p')
Budget = side_column_tags[len(side_column_tags)-2].text.strip().split(' ')[1]

In [15]:
# Revenue of movie
Revenue = side_column_tags[len(side_column_tags)-1].text.strip().split(' ')[1]

In [16]:
# Language of movie
Language = side_column_tags[len(side_column_tags)-3].text.strip().split(' ')[2]

In [17]:
Description = []
Movie_runtime = []
Budget = []
Revenue = []
Language = []

In [22]:
for i in range (0,10000) :
 
     Movie_url = movie_links[i]
 
     r = requests.get(Movie_url, headers = headers)
 
     movie_docs = BeautifulSoup(r.text, 'html.parser')
 
     desc_tag = movie_docs.find_all('div', {'class' : 'overview'})
     Description.append(desc_tag[0].text.strip())
 
     runtime_tag = movie_docs.find_all('span', {'class' : 'runtime'})
     Movie_runtime.append(runtime_tag[0].text.strip())
 
     side_column_tags = movie_docs.find('section', {'class' : 'facts left_column'}).find_all('p')
 
     Budget.append(side_column_tags[len(side_column_tags)-2].text.strip().split(' ')[1])
 
     Revenue.append(side_column_tags[len(side_column_tags)-1].text.strip().split(' ')[1])
    
     Language.append(side_column_tags[len(side_column_tags)-3].text.strip().split(' ')[2])


In [23]:
len(Language)

10000

In [32]:
dict = {
    'Movie name' : Movies_names,
    'Release date' : release_year,
    'Language' : Language,
    'Rating' : rating,
    'Runtime' : Movie_runtime,
    'Description' : Description,
    'Budget' : Budget,
    'Revenue' : Revenue,
    'Movie link' : movie_links
}

data = pd.DataFrame(dict)

In [33]:
data

,Movie name,Release date,Language,Rating,Runtime,Description,Budget,Revenue,Movie link
0,Thor: Love and Thunder,"Jul 06, 2022",English,67.0,1h 59m,After his retirement is interrupted by Gorr th...,"$250,000,000.00","$662,000,000.00",https://www.themoviedb.org/movie/616037
1,Jurassic World Dominion,"Jun 01, 2022",English,71.0,2h 27m,"Four years after Isla Nublar was destroyed, di...","$165,000,000.00","$921,000,000.00",https://www.themoviedb.org/movie/507086
2,Minions: The Rise of Gru,"Jun 29, 2022",English,75.0,1h 27m,A fanboy of a supervillain supergroup known as...,"$85,000,000.00","$710,000,000.00",https://www.themoviedb.org/movie/438148
3,Top Gun: Maverick,"May 24, 2022",English,83.0,2h 11m,After more than thirty years of service as one...,"$170,000,000.00","$1,321,104,000.00",https://www.themoviedb.org/movie/361743
4,The Black Phone,"Jun 22, 2022",English,80,1h 43m,"Finney Shaw, a shy but clever 13-year-old boy,...","$18,800,000.00","$114,000,000.00",https://www.themoviedb.org/movie/756999
...,...,...,...,...,...,...,...,...,...
9995,Spider-Man: No Way Home,"Dec 15, 2021",English,81.0,2h 28m,Peter Parker is unmasked and no longer able to...,"$200,000,000.00","$1,892,000,000.00",https://www.themoviedb.org/movie/634649
9996,Shut In,"Mar 24, 2022",English,71.0,1h 30m,A young single mother is held captive along wi...,-,"$304,106.00",https://www.themoviedb.org/movie/629015
9997,Sonic the Hedgehog 2,"Mar 30, 2022",English,77.0,2h 2m,"After settling in Green Hills, Sonic is eager ...","$110,000,000.00","$401,800,000.00",https://www.themoviedb.org/movie/675353
9998,Dog,"Feb 17, 2022",English,74.0,1h 42m,An army ranger and his dog embark on a road tr...,"$20,000,000.00","$77,263,354.00",https://www.themoviedb.org/movie/626735


In [34]:
data.shape

(10000, 9)

In [35]:
# creating csv file of this data

data.to_csv('Movies_Dataset.csv')